# 加载库函数及导入文件

In [1]:
import warnings
import pathlib
warnings.filterwarnings('ignore', message="Workbook contains no default style, apply openpyxl's default", category=UserWarning)
# import pathlib
# import pandas as pd
OUTPUT_DIR_PATH = pathlib.Path(r"J:\银行分析结果")
OUTPUT_DIR_PATH

PosixPath('J:\\银行分析结果')

# 测试

## 提取excel表头和md5值

In [12]:
%%time
from hashlib import md5
acc_file = pathlib.Path(r"E:\张楠\2023赵益华927\银行\建行\第一批 赵益华等9人\1080861461690963794022777_16_1_watermark.xlsx")
# acc_file = pathlib.Path(r"E:\张楠\2023赵益华927\银行\建行\建行第二次查询7.13\1080861701689234045070165_1_1田桂芬.xlsx")
# acc_file = pathlib.Path(r"E:\张楠\2023赵益华927\银行\天津银行\滨监查询（2023）05217号.xls")
# acc_file = pathlib.Path(r"E:\张楠\2023赵益华927\银行\天津银行\滨监查询（2023）05217.xls")
print(a := read_header(acc_file))
md5(a.encode()).hexdigest()

('序号', '客户名称', '账号', '币种', '钞汇', '册号', '笔号', '交易日期', '交易时间', '活期存款账户明细号', '交易卡号', '摘要', '借贷方向', '交易金额', '账户余额', '柜员号', '交易机构号', '交易机构名称', '对方账号', '对方户名', '对方行名', '交易流水号', '交易渠道', '自助设备编号', '扩充备注', 'IP地址', 'MAC地址', '第三方订单号', '商户号', '商户名称')
Wall time: 951 ms


D:\soft\WPy64-3890\python-3.8.9.amd64\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


'8fa9c0776b857793f91335192398aeba'

## 测试yaml配置文件语法，及内部操作指令对象转换

In [35]:
%run corelibs/config.py
CONF_DATA

{'output_format': 'excel',
 'header_hash': {'6d3d0d1e252f43a000010e19d8b2dd57': ['天津银行', '账户'],
  '7509cd85808831c7004c8c78e9267d7c': ['天津银行', '流水'],
  '5ff3970075646f3a7fffdef5430539d7': ['建设银行网点', '账户'],
  '8fa9c0776b857793f91335192398aeba': ['建设银行网点', '流水']},
 '天津银行': {'账户': {'银行': '天津银行',
   '姓名': {'开户名称': True},
   '证件号码': True,
   '电话': {'短信捆绑电话': None, '网银电话': None},
   '账号': {'客户账号': True},
   '开户机构': {'开户机构名称': None},
   '销户机构': {'销户机构名称': None},
   'cols_new_order': ['银行',
    '姓名',
    '证件号码',
    '电话',
    '地址',
    '账号',
    '卡号',
    '余额',
    '开户日期',
    '开户机构',
    '账户状态',
    '销户日期',
    '销户机构',
    '销户机构号',
    '所属银行',
    '是否开通网易',
    'U盾号',
    '邮编',
    '短信捆绑电话',
    '网银电话']},
  '流水': {'银行': '天津银行',
   '日期': ['date', '交易日期'],
   '时间': ['time', '交易日期'],
   '姓名': {'我方中文名称': True},
   '账号': {'我行账号（卡号）': True},
   '摘要': {'用途（摘要）': None},
   '出账金额': ['C', '借贷标志', '借', '交易金额'],
   '入账金额': ['D', '借贷标志', '贷', '交易金额'],
   '余额': 100,
   '交易机构': {'受理机构名称': None, '我方银行名称': No

In [36]:
conf_obj = get_conf_obj('天津银行','流水')
[print(x) for x in conf_obj]

None
{'银行': '天津银行'}
{'我方中文名称': True, '我行账号（卡号）': True, '余额': True, '借贷标志': True, '交易金额': True, '交易日期': True}
{'我方中文名称': '姓名', '我行账号（卡号）': '账号', '用途（摘要）': '摘要', '对方中文名称': '对方户名', '对方账号（卡号）': '对方账号', '对方银行名称': '对方开户行', '网银IP地址': 'IP地址'}
{'交易机构': ['受理机构名称', '我方银行名称']}
{'日期': ['交易日期']}
{'时间': ['交易日期']}
{'余额': True, '交易金额': True}
{'C': '借', 'CD_col': '借贷标志', 'C_col': '出账金额', 'trans_col': '交易金额', 'D': '贷', 'D_col': '入账金额'}
{}
['银行', '日期', '时间', '姓名', '账号', '摘要', '币种', '出账金额', '入账金额', '余额', '交易机构', '对方账号', '对方户名', '对方开户行', 'IP地址', '我方银行名称', '对方银行机构代码', '受理机构代码', '交易日期', '借贷标志', '交易金额', '身份证号', '流水号', '交易代码（交易渠道）', '代码描述', '交易是否成功']


[None, None, None, None, None, None, None, None, None, None, None]

In [ ]:
CONF_TPL_CACHE

##  测试单个账户文件读取及储存

In [5]:
%run corelibs.py
acc_file = pathlib.Path(r"E:\张楠\2023赵益华927\银行\建行\第一批 赵益华等9人\1080861461690963794022777_15_1时燕.xlsx")
acc_file = pathlib.Path(r"E:\张楠\2023赵益华927\银行\天津银行\滨监查询（2023）05217.xls")
conf_obj = get_conf_obj('天津银行', '账户')
df = parse_sheet_general(acc_file, conf_obj)
save_accounts(df, OUTPUT_DIR_PATH, '天津银行')

4

In [3]:
import dtale

In [4]:
dtale.show(df, open_browser=True)

## 测试单个流水文件的读取与储存

In [7]:
%run corelibs.py
acc_file = pathlib.Path(r"E:\张楠\2023赵益华927\银行\建行\第一批 赵益华等9人\1080861461690963794022777_16_1_watermark.xlsx")
conf_obj = get_conf_obj('建设银行网点', '流水')
df = parse_sheet_general(acc_file, conf_obj)
grouped = df.groupby('账号')
df_list = [x.reset_index(drop=True) for _ , x in grouped]
save_statements(df_list, OUTPUT_DIR_PATH, '建设银行网点', '1111111-11')

D:\soft\WPy64-31160\notebooks\pynbCaseData\corelibs.py:119: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[_k] = pd.to_datetime(df[_v]).dt.time


9999

In [15]:
import dtale

In [28]:
dtale.show(df, open_browser=True)

# 执行

## 分析建设银行网点数据目录
（如：开发区金融街建行查询数据）

In [9]:
%run corelibs/banks.py
base_dir = pathlib.Path(r"J:\杨孟军\银行\建行\建行(于东林、刘德阳等)")
err_files = process_dir_ccb_branch(base_dir, OUTPUT_DIR_PATH, '五室调取')
print('以下为发生错误而跳过的文件：')
len([print(k.name + v) for k, v in err_files.items()])

  0%|          | 0/8 [00:00<?, ?it/s]

1080860401675667712130255_1_1_watermark.xlsx……建设银行网点账户done
1080860401675667712130255_2_1_watermark.xlsx……

D:\WPy64-31160\notebooks\pynbCaseData\corelibs\data.py:23: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[_k] = pd.to_datetime(df[_v]).dt.time


建设银行网点流水done
1080860401675667712130255_3_1_watermark.xlsx……建设银行网点账户done
1080860401675667712130255_4_1_watermark.xlsx……

D:\WPy64-31160\notebooks\pynbCaseData\corelibs\data.py:23: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[_k] = pd.to_datetime(df[_v]).dt.time


...

D:\WPy64-31160\notebooks\pynbCaseData\corelibs\data.py:23: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[_k] = pd.to_datetime(df[_v]).dt.time


建设银行网点流水done
1080860401675667712130255_7_1_watermark.xlsx……建设银行网点账户done
1080860401675667712130255_8_1_watermark.xlsx……

D:\WPy64-31160\notebooks\pynbCaseData\corelibs\data.py:23: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[_k] = pd.to_datetime(df[_v]).dt.time


建设银行网点流水done
以下为发生错误而跳过的文件：


0

# 杂项

In [48]:
pd.read_excel

<function pandas.io.excel._base.read_excel(io, sheet_name=0, header=0, names=None, index_col=None, usecols=None, squeeze=False, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skiprows=None, nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, parse_dates=False, date_parser=None, thousands=None, comment=None, skipfooter=0, convert_float=True, mangle_dupe_cols=True, storage_options: Union[Dict[str, Any], NoneType] = None)>

In [6]:
hash(str(['所属银行','证件号码','开户名称','客户账号','卡号','余额','开户日期','开户机构名称','是否开通网易','U盾号','短信捆绑电话','网银电话','地址','邮编','销户日期','销户机构号','销户机构名称','账户状态','我行账号（卡号）','我方银行名称','我方中文名称','身份证号','交易日期','对方账号（卡号）','对方中文名称','对方银行名称','对方银行机构代码','流水号','借贷标志','交易金额','币种','余额','网银IP地址','交易代码（交易渠道）','代码描述','用途（摘要）','受理机构代码','受理机构名称','交易是否成功']))

3955050809677091182

In [124]:
type(CONF_DATA['天津银行']['流水']['时间']['交易日期'])

TypeError: list indices must be integers or slices, not str

In [ ]:
CONF_DATA['天津银行']['流水']

In [27]:
print(CONF_DATA.get('哈哈'))

In [37]:
from itertools import islice
a = [3,4,5,6,7,8]
b = islice(a,4,5)
next(b)

In [8]:
import pandas as pd
pd.to_datetime('2023-01-01', format='mixed')

Timestamp('2023-01-01 00:00:00')

['__add__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_asdict',
 '_field_defaults',
 '_fields',
 '_make',
 '_replace',
 'bank_name',
 'cdid',
 'col_name_map',
 'cols_new_order',
 'count',
 'date_cols',
 'digi_cols',
 'fill_cols',
 'index',
 'merge_2cols',
 'new_cols',
 'time_cols',
 'verify_cols']